In [1]:
from pathlib import Path

import pandas as pd
from dotenv import load_dotenv

import pipeline  # our local pipeline.py

# Load .env so OPENAI_API_KEY is available
load_dotenv()

# Detect project root by climbing until we see data/
ROOT_DIR = Path.cwd()
for _ in range(6):
    if (ROOT_DIR / "data").exists():
        break
    ROOT_DIR = ROOT_DIR.parent

print("Detected project root:", ROOT_DIR)

INPUT_JSONL = ROOT_DIR / "data" / "raw" / "synthetic" / "issue_tracking" / "synthetic_whatsapp_client_messages.jsonl"
OUTPUT_DIR = ROOT_DIR / "notebooks" / "issue_tracking" / "cel" / "results"

INPUT_JSONL, OUTPUT_DIR

Detected project root: c:\Users\tengc\Downloads\develop_ai_pipelines_testing\ai_pipeline_testing


(WindowsPath('c:/Users/tengc/Downloads/develop_ai_pipelines_testing/ai_pipeline_testing/data/raw/synthetic/issue_tracking/synthetic_whatsapp_client_messages.jsonl'),
 WindowsPath('c:/Users/tengc/Downloads/develop_ai_pipelines_testing/ai_pipeline_testing/notebooks/issue_tracking/cel/results'))

In [2]:
from importlib import reload

reload(pipeline)  # so edits to pipeline.py are picked up

results_df = pipeline.run_issue_tracking_pipeline(
    input_path=INPUT_JSONL,
    output_dir=OUTPUT_DIR,
    model="gpt-4.1-mini",
    test_mode=True,      # change to False for full run
    max_rows=30,
    sleep_sec=0.0,
)

results_df.head()

Reading input from: c:\Users\tengc\Downloads\develop_ai_pipelines_testing\ai_pipeline_testing\data\raw\synthetic\issue_tracking\synthetic_whatsapp_client_messages.jsonl
TEST_MODE=True → processing first 30 rows only
[1/30] Processing message_id=WAM_001 ...
[2/30] Processing message_id=WAM_003 ...
[3/30] Processing message_id=WAM_005 ...
[4/30] Processing message_id=WAM_006 ...
[5/30] Processing message_id=WAM_008 ...
[6/30] Processing message_id=WAM_010 ...
[7/30] Processing message_id=WAM_011 ...
[8/30] Processing message_id=WAM_013 ...
[9/30] Processing message_id=WAM_015 ...
[10/30] Processing message_id=WAM_017 ...
[11/30] Processing message_id=WAM_018 ...
[12/30] Processing message_id=WAM_020 ...
[13/30] Processing message_id=WAM_022 ...
[14/30] Processing message_id=WAM_024 ...
[15/30] Processing message_id=WAM_026 ...
[16/30] Processing message_id=WAM_028 ...
[17/30] Processing message_id=WAM_029 ...
[18/30] Processing message_id=WAM_031 ...
[19/30] Processing message_id=WAM_033

,chat_id,message_id,timestamp,sender_number,sender_name,message_text,issue,customer,created_by,created_at,to_inform,assigned_to,resolved_at,status_to_close,closed_at,resolution_score,comments,raw_model_output,error
0,65001,WAM_001,2024-11-12 14:23:45,6591234567,BRC Asia,"Hi good afternoon, may I know what time the go...",Delivery Delay,BRC Asia,Customer,2024-11-12 14:23:45,"AM, Cust Svc, Site",Cust Svc,,Customer,,Not resolved yet,Customer enquiring delivery time; check ETA an...,"{\n ""issue"": ""Delivery Delay"",\n ""customer"":...",None
1,65002,WAM_003,2024-11-13 09:15:33,6587654321,Hup Seng,Boss ask already — when can deliver? Urgent leh.,Delivery Delay,Hup Seng,Customer,2024-11-13 09:15:33,"AM, Cust Svc",Cust Svc,,Customer,,Not resolved yet,Customer urgently requesting delivery ETA; fol...,"{\n ""issue"": ""Delivery Delay"",\n ""customer"":...",None
2,65002,WAM_005,2024-11-13 09:30:12,6587654321,Hup Seng,Ok tks.,Other,Hup Seng,Customer,2024-11-13 09:30:12,,,,,,Not resolved yet,Acknowledgement message from customer; no acti...,"{\n ""issue"": ""Other"",\n ""customer"": ""Hup Sen...",None
3,65003,WAM_006,2024-11-13 10:42:19,6593456789,Soon Lee,"Morning, can help check stock for SS pipe 4 in...",Stock Issues,Soon Lee,Customer,2024-11-13 10:42:19,"Purchase, Cust Svc",Purchase,,Customer,,Not resolved yet,Customer requesting stock check for 4 inch SS ...,"{\n ""issue"": ""Stock Issues"",\n ""customer"": ""...",None
4,65004,WAM_008,2024-11-13 13:28:54,6581234567,Kim Heng,Bro the item you all send wrong leh. I ordered...,Fulfilment Error,Kim Heng,Customer,2024-11-13 13:28:54,"AM, Cust Svc",Cust Svc,,Customer,,Not resolved yet,Customer received wrong item size; verify orde...,"{\n ""issue"": ""Fulfilment Error"",\n ""customer...",None


In [3]:
# How many had parsing/LLM errors?
print("Total rows:", len(results_df))
print("Errors:", results_df["error"].notna().sum())

# Quick look at issue category distribution
results_df["issue"].value_counts(dropna=False)

Total rows: 30
Errors: 0


issue
Delivery Delay      8
Other               6
Stock Issues        6
Fulfilment Error    3
Product Quality     3
Payments            2
Service Issues      2
Name: count, dtype: int64

In [4]:
# See rows with errors (if any)
results_df[results_df["error"].notna()][[
    "chat_id",
    "message_id",
    "message_text",
    "raw_model_output",
    "error",
]].head()

# See a sample mapping from raw WhatsApp message to issue row
cols = [
    "chat_id",
    "message_id",
    "timestamp",
    "sender_name",
    "message_text",
    "issue",
    "assigned_to",
    "to_inform",
    "resolution_score",
    "comments",
]
results_df[cols].head(10)

,chat_id,message_id,timestamp,sender_name,message_text,issue,assigned_to,to_inform,resolution_score,comments
0,65001,WAM_001,2024-11-12 14:23:45,BRC Asia,"Hi good afternoon, may I know what time the go...",Delivery Delay,Cust Svc,"AM, Cust Svc, Site",Not resolved yet,Customer enquiring delivery time; check ETA an...
1,65002,WAM_003,2024-11-13 09:15:33,Hup Seng,Boss ask already — when can deliver? Urgent leh.,Delivery Delay,Cust Svc,"AM, Cust Svc",Not resolved yet,Customer urgently requesting delivery ETA; fol...
2,65002,WAM_005,2024-11-13 09:30:12,Hup Seng,Ok tks.,Other,,,Not resolved yet,Acknowledgement message from customer; no acti...
3,65003,WAM_006,2024-11-13 10:42:19,Soon Lee,"Morning, can help check stock for SS pipe 4 in...",Stock Issues,Purchase,"Purchase, Cust Svc",Not resolved yet,Customer requesting stock check for 4 inch SS ...
4,65004,WAM_008,2024-11-13 13:28:54,Kim Heng,Bro the item you all send wrong leh. I ordered...,Fulfilment Error,Cust Svc,"AM, Cust Svc",Not resolved yet,Customer received wrong item size; verify orde...
5,65004,WAM_010,2024-11-13 13:47:08,Kim Heng,Today must deliver ah. Tmr got inspection alre...,Delivery Delay,Cust Svc,"AM, Cust Svc",Not resolved yet,Customer urgent delivery request due to inspec...
6,65005,WAM_011,2024-11-13 15:12:37,Lian Huat Trading,"Hi, your driver called say the place very ulu,...",Delivery Delay,Cust Svc,"AM, Cust Svc, Site",Not resolved yet,Customer reports driver unable to find deliver...
7,65006,WAM_013,2024-11-14 08:34:21,Tai Seng Hardware,"Morning Boss, still waiting for quotation leh....",Other,AM,AM,Not resolved yet,Customer following up on pending quotation sen...
8,65007,WAM_015,2024-11-14 10:47:18,Metalform Industries,Bro urgent need. 3 inch reducer 10 pcs. Can de...,Stock Issues,Purchase,"Purchase, Cust Svc",Not resolved yet,Customer urgently needs 3 inch reducer 10 pcs;...
9,65007,WAM_017,2024-11-14 11:05:14,Metalform Industries,Must be morning lah. Our contractor very fierc...,Other,AM,AM,Not resolved yet,Customer insists delivery must be in the morni...
